In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pytesseract

import os
import subprocess
from pathlib import Path

from maddrive_adas.sign_det.base import AbstractSignClassifier
from maddrive_adas.sign_det.classifier import EncoderBasedClassifier
from maddrive_adas.utils.fs import imread_rgb

try:
    output = subprocess.check_output(
        'tesseract -v',
        stderr=subprocess.STDOUT
    ).decode()
    if 'tesseract' not in output:
        raise subprocess.CalledProcessError
except subprocess.CalledProcessError:
    print('Unable to call tessecact. Install and add tesseract to PATH variable.')
    print('Link: https://tesseract-ocr.github.io/tessdoc/Downloads.html')
    raise subprocess.CalledProcessError

def crop_img(img, xscale=1.0, yscale=1.0):
    center_x, center_y = img.shape[1] / 2, img.shape[0] / 2
    width_scaled, height_scaled = img.shape[1] * xscale, img.shape[0] * yscale
    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
    img_cropped = img[int(top_y):int(bottom_y), int(left_x):int(right_x)]
    return img_cropped

try:
    PROJECT_ROOT = Path(os.readlink(f'/proc/{os.environ["JPY_PARENT_PID"]}/cwd'))
except FileNotFoundError:
    __file = %pwd
    PROJECT_ROOT = Path(__file).parents[1]
    
data_path: Path = Path(PROJECT_ROOT / 'SignDetectorAndClassifier/data/additional_sign')
CLASSIFIER_ARCHIVE = PROJECT_ROOT / 'encoder_archive'

c: AbstractSignClassifier = EncoderBasedClassifier(
    config_path=str(CLASSIFIER_ARCHIVE)
)

In [ ]:
import matplotlib

In [ ]:
for file in data_path.iterdir():
    if '3.24' in str(file) or '3.25' in str(file):
        img = imread_rgb(file)
        res, d = c._fixup_signs_with_text(img, ('3.24', 1.), ret_debug_img=True)
        fig, ax = plt.subplots(1, len(d) + 1, figsize=(4,4))
        # print(len(d))
        if isinstance(ax, np.ndarray):
            # print('mult')
            ax[0].title.set_text(res[0])
            ax[0].imshow(img)
            for i in range(0, len(d)):
                # print(i)
                ax[i+1].imshow(d[i], cmap='gray')
        else:
            ax.title.set_text(res[0])
            ax.imshow(img)
        
plt.show()

In [ ]:
list(int(scale / x) for x in img.shape)